# Link Prediction using Collaborative Filtering
In a previous [project](https://github.com/chauvu/chauvu.github.io/blob/main/Notebooks/graph_link_prediction.ipynb), I have demonstrated the use of Graph Convolutional Network (GCN) to perform novel link predictions in knowledge graphs. Other prominent works have also shown that graph embedding models (such as translational distance models like TransE that encode both entities and relations as low-dimensional vectors) are able to perform link prediction with robust results.

In this current project, instead of applying learned embeddings (such as [TransE](https://proceedings.neurips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf) embeddings or even [Node2Vec](https://snap.stanford.edu/node2vec/) embeddings), I will apply Collaborative Filter to perform link prediction. Collaborative Filtering is a technique frequently used for constructing recommendation systems. For example, Amazon can use collaborative filtering to recommend potential products a particular user is interested in based on the products that user has viewed/liked/purchased in the past. 

To apply collaborative filter to drug repurposing (predicting novel links between drug and disease nodes in a knowledge graph), I will view the drug nodes as the Amazon's users and the disease nodes as the potential products for purchase. I will create a sparse utility matrix with binary values: 1 encodes an existing link and 0 encodes no link between the particular drug-disease pair. Link prediction scores are generated from the dot product of the drug vector and disease vector. To draw a parallel to knowledge graph completion works, I will use KG evaluation metrics: *mean rank* and *hits@k*. I will also be working with a well-known biomedical knowledge graph dataset: [Hetionet](https://het.io/about/). 

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from pykeen.datasets import Hetionet
import networkx as nx
import pandas as pd
import numpy as np
import os

### Dataset
Instead of downloading Hetionet from its original [website](https://het.io/about/), I will be using the default dataset in [PyKEEN](https://github.com/pykeen/pykeen). This dataset is already in triple format. Below is the code that I used in the previous project on link prediction [here](https://github.com/chauvu/chauvu.github.io/blob/main/Notebooks/graph_link_prediction.ipynb) that processed the Hetionet data from PyKEEN.

In [2]:
meta_edge = {
				'AdG': 'Anatomy-downregulates-Gene',
				'AeG': 'Anatomy-expresses-Gene',
				'AuG': 'Anatomy-upregulates-Gene',
				'CbG': 'Compound-binds-Gene',
				'CcSE': 'Compound-causes-Side Effect',
				'CdG': 'Compound-downregulates-Gene',
				'CpD': 'Compound-palliates-Disease',
				'CrC': 'Compound-resembles-Compound',
				'CtD': 'Compound-treats-Disease',
				'CuG': 'Compound-upregulates-Gene',
				'DaG': 'Disease-associates-Gene',
				'DdG': 'Disease-downregulates-Gene',
				'DlA': 'Disease-localizes-Anatomy',
				'DpS': 'Disease-presents-Symptom',
				'DrD': 'Disease-resembles-Disease',
				'DuG': 'Disease-upregulates-Gene',
				'GcG': 'Gene-covaries-Gene',
				'GiG': 'Gene-interacts-Gene',
				'GpBP': 'Gene-participates-Biological Process',
				'GpCC': 'Gene-participates-Cellular Component',
				'GpMF': 'Gene-participates-Molecular Function',
				'GpPW': 'Gene-participates-Pathway',
				'Gr>G': 'Gene-regulates-Gene',
				'PCiC': 'Pharmacologic Class-includes-Compound',
			}

data = Hetionet(create_inverse_triples=False, random_state=357)
training = data.training
validation = data.validation
testing = data.testing

training_df = training.tensor_to_df(training.mapped_triples)[['head_label', 'relation_label', 'tail_label']]
training_df = training_df.rename(columns={'head_label': 'head', 'relation_label': 'edge', 'tail_label': 'tail'})
validation_df = validation.tensor_to_df(validation.mapped_triples)[['head_label', 'relation_label', 'tail_label']]
validation_df = validation_df.rename(columns={'head_label': 'head', 'relation_label': 'edge', 'tail_label': 'tail'})
testing_df = testing.tensor_to_df(testing.mapped_triples)[['head_label', 'relation_label', 'tail_label']]
testing_df = testing_df.rename(columns={'head_label': 'head', 'relation_label': 'edge', 'tail_label': 'tail'})

data_df = pd.concat([training_df, validation_df, testing_df])
data_df['edge_class'] = data_df['edge'].map(meta_edge)
data_df['head_class'] = data_df['edge_class'].str.split('-').apply(lambda x: x[0])
data_df['tail_class'] = data_df['edge_class'].str.split('-').apply(lambda x: x[-1])

In [3]:
data_df.head()

,head,edge,tail,edge_class,head_class,tail_class
0,Anatomy::UBERON:0000002,AdG,Gene::10005,Anatomy-downregulates-Gene,Anatomy,Gene
1,Anatomy::UBERON:0000002,AdG,Gene::114804,Anatomy-downregulates-Gene,Anatomy,Gene
2,Anatomy::UBERON:0000002,AdG,Gene::118670,Anatomy-downregulates-Gene,Anatomy,Gene
3,Anatomy::UBERON:0000002,AdG,Gene::128989,Anatomy-downregulates-Gene,Anatomy,Gene
4,Anatomy::UBERON:0000002,AdG,Gene::132851,Anatomy-downregulates-Gene,Anatomy,Gene


In [4]:
data_df.shape

(2250197, 6)

The Hetionet dataset has approximately 2 million triples, with different entity types such as anatomy, gene, drug, disease, etc. In this work, I am only interested in the links between *drug* (compound) and *disease* entities. And I do not care what relation types are between these nodes, such as `Compound-palliates-Disease` or `Compound-treats-Disease`. After filtering all other triple types out, I end up with 1,145 triples of drug-disease type.

In [5]:
data_df = data_df[(data_df['head_class']=='Compound') & (data_df['tail_class']=='Disease')][['head', 'tail']]

In [6]:
data_df.shape

(1145, 2)

Afterward, I will perform train-test-split (90/10), ensuring that all entities in the test set have appeared in the training set. This ensured a static utility matrix since collaborative filter does not perform well on unseen entities, such as when a new user joins Amazon with no history of viewing/purchasing any products.

In [7]:
data_train, data_test = train_test_split(data_df, test_size=0.1, random_state=100)
entities_train = set(list(data_train['head'].unique()) + list(data_train['tail'].unique()))
entities_test = set(list(data_test['head'].unique()) + list(data_test['tail'].unique()))
entities_test_diff = entities_test - entities_train
data_test_head_diff = data_test['head'].apply(lambda x: x in entities_test_diff)
data_test_tail_diff = data_test['tail'].apply(lambda x: x in entities_test_diff)
data_test_diff = data_test[data_test_head_diff | data_test_tail_diff]
data_test = data_test[~data_test_head_diff & ~data_test_tail_diff]
data_train = data_train.append(data_test_diff).drop_duplicates(ignore_index=True)

### Utility Matrix
A utility matrix represents links between the users and the products. In this work, I will create an N x M utility matrix, where `N=551` is the number of unique drug entities and `M=91` is the number of unique diseases. Since each entity is represented by its name (e.g. `Compound::DB00014`) or its index (integer number), I will create dictionaries to convert between the names and indices.

In [8]:
head_entities = data_train['head'].unique().tolist()
tail_entities = data_train['tail'].unique().tolist()

In [9]:
head_index2entity_dict = {index: entity for index, entity in enumerate(head_entities)}
head_entity2index_dict = {entity: index for index, entity in enumerate(head_entities)}
tail_index2entity_dict = {index: entity for index, entity in enumerate(tail_entities)}
tail_entity2index_dict = {entity: index for index, entity in enumerate(tail_entities)}

In [10]:
utility_matrix = np.zeros((len(head_entities), len(tail_entities)))
for index, row in data_train.iterrows():
	utility_matrix[head_entity2index_dict[row['head']], tail_entity2index_dict[row['tail']]] = 1

In [11]:
utility_matrix.shape

(551, 91)

For example, compound node of `compound_index=0` (`Compound::DB00755`) only has 1 link to a disease node of `disease_index=0` (`Disease::DOID:1192`). This is well represented in this compound's vector `utility_matrix[0,:]` shown below.

In [12]:
print(head_index2entity_dict[0])
print(tail_index2entity_dict[0])
utility_matrix[0,:]

Compound::DB00755
Disease::DOID:1192


array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

### Score Matrix
To compute the similarity between two compound nodes, I will calculate the *dot product* of their representation vectors from the utility matrix. Since this process will need to be repeated for every prediction for the test set, there will be a lot of overlap, so this score matrix of size N x N (551x551) should be pre-computed.

In [13]:
score_function = np.dot 

In [14]:
score_matrix = np.zeros((len(head_entities), len(head_entities)))
for h1_index, h1 in enumerate(head_entities):
	for h2_index, h2 in enumerate(head_entities):
		if h2_index >= h1_index: # top triu
			vector_h1 = utility_matrix[h1_index, :]
			vector_h2 = utility_matrix[h2_index, :]
			score = score_function(vector_h1, vector_h2)
			score_matrix[h1_index, h2_index] = score
			score_matrix[h2_index, h1_index] = score

In [15]:
score_matrix.shape

(551, 551)

Here is a small portion of our score matrix, we notice that the diagonal line always has large values since this represents the dot product of a compound's vector with itself. Therefore, for a compound, I will need to take special care to filter out itself from its cluster of most similar nodes.

In [16]:
score_matrix[:5,:5]

array([[ 1.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0., 10.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

### Collaborative Filtering
In this portion, I implement collaborative filtering. For each test triple in the test set, I will use the pre-computed score matrix above to generate the top `k=3` nodes that are most similar to the head entity (except itself).

Afterward, I will implement a majority voting scheme, in which all disease nodes that have connections to the top `k=3` most simlar drug nodes are considered. The votes are the number of links a particular disease has with the `k` drug nodes, and the ranks are the index of ground-truth tail the descending-sorted votes. I also took special care to filter out triples that are already known to be true (from the training set to avoid artificially inflated evaluation metrics).

In [17]:
k = 3 # top k similar nodes

In [18]:
ranks = [] # list of ranks of ground-truths for test set

In [19]:
for index_test in range(len(data_test)):
	triple_test = data_test.iloc[index_test]

	head_test = triple_test['head']
	head_index = head_entity2index_dict[head_test]
	tail_test = triple_test['tail']
	tail_index = tail_entity2index_dict[tail_test]

	scores = score_matrix[head_index, :]
	similar_heads_index = list(np.argsort(scores)[::-1][:k+1]) # descending, top k
	if head_index in similar_heads_index: # filter out itself
		similar_heads_index.remove(head_index)
	else: similar_heads_index = similar_heads_index[:k]

	# majority voting
	head_utility_value = np.zeros(len(tail_entities))
	for head_similar_index in similar_heads_index:
		head_similar = head_index2entity_dict[head_similar_index]
		head_utility_value += utility_matrix[head_similar_index, :]

	ranked_tails_index = list(np.argsort(head_utility_value)[::-1])

	# filter out known triples
	tails_index_to_remove = []
	for ranked_tail_index in ranked_tails_index:
		if utility_matrix[head_index, ranked_tail_index] > 0:
			tails_index_to_remove.append(ranked_tail_index)
	for ranked_tail_index in tails_index_to_remove:
		ranked_tails_index.remove(ranked_tail_index)
	
	# ground truth rank
	rank = ranked_tails_index.index(tail_index)
	ranks.append(rank)

Here, I show the ground-truth ranks for the first 10 entries of the test set. We can see that for the 3rd sample, the ground-truth answer is the top answer giving by our collaborative filter system.

In [20]:
ranks[:10]

[54, 46, 0, 14, 8, 53, 4, 39, 20, 0]

### Evaluation Metrics
In this project, I will make use of common knowledge graph evaluation metrics:
* Mean Rank (MR): the average rank of our ground-truths, perfect system yields MR=0
* Hits@k: the percentage of test set that yields the ground-truth in the top k answers, perfect system yields hits@k=1

In [21]:
mr = np.mean(ranks) # mean rank ~17
hits5 = (np.array(ranks)<5).sum() / len(ranks) # hits @ 5 ~ 0.45
hits10 = (np.array(ranks)<10).sum() / len(ranks) # hits @ 10 ~ 0.60


print('MR', mr)
print('Hits@5', hits5)
print('Hits@10', hits10)

MR 17.012048192771083
Hits@5 0.4578313253012048
Hits@10 0.6024096385542169


# Conclusion
In conclusion, this project shows that a simple collaborative filter recommendation system can predict whether a link exists between drugs and diseases reasonably in the Hetionet dataset. The hits@10 is 0.60, which is very high for a biomedical knowledge graph. However, we have to recognize that this is a very small dataset, since we have already filtered out all other triples except drug-disease relations.

One way to improve performance is that instead of constructing the utility matrix upon only drugs and diseases, it can be constructed with other types of entities nodes as well; the prediction process is subsequently limited only to disease nodes. Additionally, most knowledge graph evaluation yields *optimistic* and *pessimistic* evaluations, in which sometimes potential triples are score equally and the ground-truth is evaluated either as the highest or lowest rank of all equal answers. More pessimistic answers are probably more accurate for our case in which the matrix is extremely sparse and there are many nodes that are similar to one another, which can worsen the results.

In any case, collaborative filtering is a very simple technique that has yield reasonable results for knowledge graph link predictions. Therefore, this result should serve as the baseline to compare against more sophisticated techniques for knowledge graph completion, especially for biomedical drug repurposing applications, in the future.